In [10]:
import torch
import torchvision
from avalanche.models import IncrementalClassifier, MultiHeadClassifier, MultiTaskModule
from Scaled_KAN import *

device = "cuda" if torch.cuda.is_available() else "cpu"

################################
# DEFINE THE MODELS TO BE USED #
################################

model_KAN = FastKAN(layers_hidden=[28*28,32,10], num_grids=4, device=device)
model_incremental = IncrementalClassifier(in_features=28*28)
model_multihead = MultiHeadClassifier(in_features=28*28)
# model_multi_task = MultiTaskModule(in_features=28*28)

In [23]:
from avalanche.benchmarks.datasets import MNIST
from avalanche.benchmarks.datasets.torchvision_wrapper import CIFAR100
from avalanche.benchmarks.datasets.dataset_utils import default_dataset_location
from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_from_datasets
from avalanche.benchmarks.utils import as_classification_dataset, AvalancheDataset

datadir = default_dataset_location('mnist')
train_MNIST = as_classification_dataset(MNIST(datadir, train=True, download=True))
test_MNIST = as_classification_dataset(MNIST(datadir, train=False, download=True))

datadir = default_dataset_location('cifar100')
train_CIFAR100 = as_classification_dataset(CIFAR100(datadir, train=True, download=True))
test_CIFAR100 = as_classification_dataset(CIFAR100(datadir, train=False, download=True))


benchmark = benchmark_from_datasets(
    train=[train_MNIST, train_CIFAR100],
    test=[test_MNIST, test_CIFAR100]
)

print(f"{benchmark.train_stream.name} - len {len(benchmark.train_stream)}")
print(f"{benchmark.test_stream.name} - len {len(benchmark.test_stream)}")

# transformations are managed by the AvalancheDataset
train_transforms = torchvision.transforms.ToTensor()
eval_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((32, 32))
])

train_MNIST = as_classification_dataset(
    train_MNIST,
    transform_groups={
        'train': train_transforms, 
        'eval': eval_transforms
    }
)
test_MNIST = as_classification_dataset(
    test_MNIST,
    transform_groups={
        'train': train_transforms, 
        'eval': eval_transforms
    }
)




Files already downloaded and verified
Files already downloaded and verified
train - len 2
test - len 2


In [24]:
################################
#   ITERATE OVER THE DATASET   #
################################

print(train_MNIST[0])

#  Using Pytorch DataLoader
train_loader = torch.utils.data.DataLoader(
    train_MNIST, batch_size=32, shuffle=True
)

for i, (x, y) in enumerate(train_loader):
    print(i)
    # print(f"Batch {i}: {x.shape} - {y.shape}")
    break

# we can also switch between train/eval transforms
train_MNIST.train()
print(train_MNIST[0][0].shape)

train_MNIST.eval()
print(train_MNIST[0][0].shape)

(<PIL.Image.Image image mode=L size=28x28 at 0x716BE5F451C0>, 5)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [41]:
from avalanche.benchmarks.classic import SplitMNIST

bechmark1 = SplitMNIST(
    n_experiences=5,
    return_task_id=True,  # add task labels
    seed=1
)

print(f'--- Stream: {bechmark1.train_stream.name}')

for exp in bechmark1.train_stream:
    eid = exp.current_experience # experience id
    clss = exp.classes_in_this_experience # classes in this experience
    tls = exp.task_labels # task labels
    # print(f"EID={eid}, classes={clss}, tasks={tls}")
    # print(f"data: {len(exp.dataset)} samples")

for exp in bechmark1.test_stream:
    print(f"EID={exp.current_experience}, classes={exp.classes_in_this_experience}, task={tls}")


--- Stream: train
EID=0, classes=[5, 6], task=[4]
EID=1, classes=[1, 2], task=[4]
EID=2, classes=[0, 8], task=[4]
EID=3, classes=[9, 3], task=[4]
EID=4, classes=[4, 7], task=[4]
